In [1]:
import pandas as pd
import numpy as np
import sklearn 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
ds = pd.read_csv("D:/ML/Leaf_Classification/train.csv.zip")
ds_test = pd.read_csv("D:/ML/Leaf_Classification/test.csv.zip")

In [3]:
X_train = ds.drop(['species', 'id'], axis=1) 
Y_train = ds['species']

In [4]:
X_test = ds_test.drop([ 'id'], axis=1) 

In [5]:
scaler = StandardScaler()
scaler.fit(X_train, Y_train)
scaled_train_data = scaler.transform(X_train)
scaled_test_data = scaler.transform(X_test)


In [6]:
import xgboost as xgb 

In [7]:
model = xgb.XGBClassifier()

In [8]:

model.fit(scaled_train_data, Y_train)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [9]:
model.get_params().keys()

dict_keys(['missing', 'max_depth', 'scale_pos_weight', 'max_delta_step', 'silent', 'learning_rate', 'seed', 'objective', 'base_score', 'min_child_weight', 'nthread', 'reg_alpha', 'colsample_bytree', 'colsample_bylevel', 'reg_lambda', 'gamma', 'subsample', 'n_estimators'])

In [10]:
parameters_grid = {
    'max_depth':range(1,10,1),
    #'gamma' : np.linspace(0,10,num=21),
    #'subsample': np.linspace(0,1,num=21),
    #'min_child_weight':np.linspace(0,20,num=21),
    #'colsample_bytree':range(1,10,1),
    #'eta':np.linspace(0,1)
}

In [11]:
grid_cv = GridSearchCV(model, parameters_grid, scoring = 'accuracy',cv=5)

In [12]:
%%time
grid_cv.fit(scaled_train_data,Y_train) 

Wall time: 1h 12min 15s


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': range(1, 10)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='accuracy', verbose=0)

In [13]:
predictions_p = grid_cv.best_estimator_.predict_proba(scaled_test_data)

In [14]:
predictions_p

array([[  6.91379746e-03,   1.51414424e-03,   1.22545427e-03, ...,
          2.26981984e-03,   1.89315286e-02,   1.80445972e-03],
       [  3.47216934e-04,   2.96987244e-04,   5.42712770e-03, ...,
          2.20464705e-03,   2.30052072e-04,   9.83443111e-04],
       [  1.17585261e-03,   7.82484591e-01,   1.51116727e-03, ...,
          2.30786041e-04,   2.86586670e-04,   8.29170737e-03],
       ..., 
       [  9.53459879e-04,   4.08868399e-03,   3.67194385e-04, ...,
          1.07910589e-03,   4.90282371e-04,   1.15296058e-03],
       [  3.81449261e-03,   1.90082623e-03,   1.07260607e-02, ...,
          1.53435357e-02,   3.02683841e-03,   6.60172384e-03],
       [  5.18890956e-05,   8.05188436e-04,   1.88002377e-04, ...,
          4.17723459e-05,   4.54610963e-05,   1.45153856e-04]], dtype=float32)

In [15]:
result = pd.DataFrame(index=ds_test.id, columns=model.classes_, data=predictions_p)
sub = result.reset_index()
sub.to_csv('submission.csv', index = False)


In [16]:
sub.tail()

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
589,1576,0.001660,0.800807,0.001581,0.003666,0.002520,0.000356,0.000568,0.003917,0.017043,...,0.000482,0.000220,0.000342,0.000949,0.001099,0.001257,0.000718,0.000241,0.000300,0.021337
590,1577,0.003932,0.000437,0.000500,0.004109,0.001149,0.000545,0.001174,0.006087,0.002117,...,0.000996,0.000598,0.003931,0.000745,0.004822,0.004459,0.002314,0.000499,0.000620,0.001444
591,1579,0.000953,0.004089,0.000367,0.000517,0.000534,0.000394,0.000546,0.003638,0.000706,...,0.000463,0.000269,0.000851,0.001910,0.000349,0.000211,0.000261,0.001079,0.000490,0.001153
592,1580,0.003814,0.001901,0.010726,0.008050,0.012568,0.003762,0.007988,0.003022,0.003162,...,0.004041,0.005751,0.002594,0.012121,0.009215,0.011143,0.002278,0.015344,0.003027,0.006602
593,1583,0.000052,0.000805,0.000188,0.000054,0.000113,0.000282,0.000115,0.000171,0.000115,...,0.000098,0.000057,0.000074,0.000089,0.000098,0.000265,0.000055,0.000042,0.000045,0.000145
